# Proje:Arama Sorguları Anomali Tespiti(Search Queries Anomaly Detection)

Arama Sorguları Anomali Algılama, arama sorgusu verilerindeki alışılmadık veya beklenmeyen kalıpları belirlemek için kullanılan bir tekniktir. Arama Sorguları Anomali Algılama görevi için izleyebileceğimiz süreç aşağıdadır:
* Bir arama motoru veya bir web sitesinin arama işlevi gibi kaynaktan geçmiş arama sorgusu verilerini toplamak.
* Arama sorgularının dağılımını, sıklığını ve fark edilebilir kalıpları veya eğilimleri anlamak için ilk analizi gerçekleştirmek.
* Anomali algılamaya yardımcı olabilecek arama sorgusu verilerinden ilgili özellikleri veya nitelikleri oluşturmak.
* Uygun bir anomali algılama algoritması seçmek.
* Yaygın yöntemler arasında Z-puanı analizi gibi istatistiksel yaklaşımlar ve Yalıtım Ormanları veya Tek Sınıflı SVM gibi makine öğrenimi algoritmaları kullanmak.
* Seçilen modeli hazırlanan veriler üzerinde eğitmek.
* Anomalileri veya aykırı değerleri belirlemek için eğitilen modeli arama sorgusu verilerine uygulamak.


Veriseti: https://statso.io/search-queries-anomalies-case-study/
<br/>
Sahip olduğumuz veri kümesi, kullanıcıları belirli bir web sitesine yönlendiren arama sorgularını ve ilişkili ölçümleri içerir. Bu veri kümesindeki sütunlar şunlardır: 

* **En Çok Aranan Sorgular:** Kullanıcılar tarafından kullanılan gerçek arama terimleri. 
* **Tıklamalar:** Kullanıcıların sorguyu kullandıktan sonra web sitesine tıkladığı zaman sayısı.
* **İzlenimler:** Web sitesinin sorgu için arama sonuçlarında göründüğü zaman sayısı. 
* **TO (Tıklama Oranı):** Tıklamaların gösterimlere oranı, sorgunun kullanıcıları web sitesine yönlendirmedeki etkinliğini gösterir. * * 
* **Pozisyon:** Web sitesinin sorgu için arama sonuçlarındaki ortalama sıralaması. 

Eldeki sorun, arama sorgularındaki anormallikleri tespit etmek için mevcut veri kümesini kullanmaktır; bu sorgular çoğunluktan önemli ölçüde farklı performans gösterir. Amaç, tıklamalar, gösterimler, TO ve arama konumu açısından düşük veya yüksek performans gösteren sorguları belirlemektir.

1. Gerekli olan kütüphaneleri içe aktarma:

In [1]:
import pandas as pd
from collections import Counter
import re
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

2. Veri Seti Yükleme: 

In [2]:
q_df = pd.read_csv('Queries.csv', encoding='utf-8')

In [3]:
q_df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,35.83%,1.61
1,thecleverprogrammer,2809,3456,81.28%,1.02
2,python projects with source code,2077,73380,2.83%,5.94
3,classification report in machine learning,2012,4959,40.57%,1.28
4,the clever programmer,1931,2528,76.38%,1.09


In [4]:
q_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Top queries  1000 non-null   object 
 1   Clicks       1000 non-null   int64  
 2   Impressions  1000 non-null   int64  
 3   CTR          1000 non-null   object 
 4   Position     1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


In [5]:
# Cleaning CTR column
q_df['CTR'] = q_df['CTR'].str.rstrip('%').astype('float') / 100

In [6]:
# Sorguları temizlemek ve kelimelere ayırmak için fonksiyon
def clean_and_split(query):
    words = re.findall(r'\b[a-zA-Z]+\b', query.lower())
    return words

# Her sorguyu kelimelere ayır ve her kelimenin frekansını say
word_counts = Counter()
for query in q_df['Top queries']:
    word_counts.update(clean_and_split(query))

word_freq_df = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Frequency'])

# Kelime frekanslarını çizme
fig = px.bar(word_freq_df, x='Word', y='Frequency', title='Arama Sorgularında En Sık Kullanılan 20 Kelime')
fig.show()

In [7]:
# Tıklamalara ve Gösterimlere Göre En Çok Aranan Sorgular
top_queries_clicks_vis = q_df.nlargest(10, 'Clicks')[['Top queries', 'Clicks']]
top_queries_impressions_vis = q_df.nlargest(10, 'Impressions')[['Top queries', 'Impressions']]

# Çizim
fig_clicks = px.bar(top_queries_clicks_vis, x='Top queries', y='Clicks', title='Tıklamalara Göre En Çok Aranan Sorgular')
fig_impressions = px.bar(top_queries_impressions_vis, x='Top queries', y='Impressions', title='Gösterimlere Göre En Çok Aranan Sorgular')
fig_clicks.show()
fig_impressions.show()

In [9]:
# En yüksek ve en düşük CTR'ye sahip sorgular
top_ctr_vis = q_df.nlargest(10, 'CTR')[['Top queries', 'CTR']]
bottom_ctr_vis = q_df.nsmallest(10, 'CTR')[['Top queries', 'CTR']]

# Plotting
fig_top_ctr = px.bar(top_ctr_vis, x='Top queries', y='CTR', title='OT\'ye Göre En İyi Sorgular')
fig_bottom_ctr = px.bar(bottom_ctr_vis, x='Top queries', y='CTR', title='OT\'ye Göre En Kötü Sorgular')
fig_top_ctr.show()
fig_bottom_ctr.show()

In [10]:
# Korelasyon matrisi görselleştirmesi
correlation_matrix = q_df[['Clicks', 'Impressions', 'CTR', 'Position']].corr()
fig_corr = px.imshow(correlation_matrix, text_auto=True, title='Korelasyon Matrix')
fig_corr.show()

2. Arama Sorgularında Anomalileri Tespit Etme

In [11]:
from sklearn.ensemble import IsolationForest

# İlgili özellikleri seçme
features = q_df[['Clicks', 'Impressions', 'CTR', 'Position']]

# Isolation Forest'ı başlatma
iso_forest = IsolationForest(n_estimators=100, contamination=0.01)  # contamination, beklenen aykırı değer oranıdır

# Modeli eğitme
iso_forest.fit(features)

# Anomalileri tahmin etme
q_df['anomaly'] = iso_forest.predict(features)

# Anomalileri filtreleme
anomalies = q_df[q_df['anomaly'] == -1]

In [12]:
anomalies[['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position']]

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,0.3583,1.61
1,thecleverprogrammer,2809,3456,0.8128,1.02
2,python projects with source code,2077,73380,0.0283,5.94
4,the clever programmer,1931,2528,0.7638,1.09
21,classification report,933,39896,0.0234,7.53
34,machine learning roadmap,708,42715,0.0166,8.97
82,r2 score,367,56322,0.0065,9.33
167,text to handwriting,222,11283,0.0197,28.52
858,water quality analysis,56,7359,0.0076,27.56
929,python turtle,52,18228,0.0029,18.75


Arama sorgusu verilerimizdeki anormallikler sadece aykırı değerler değildir. Bunlar büyüme, optimizasyon ve stratejik odaklanma için potansiyel alanların göstergeleridir. Bu anormallikler ortaya çıkan eğilimleri veya artan ilgi alanlarını yansıtır. Bu eğilimlere duyarlı kalmak, web sitesinin alaka düzeyini ve kullanıcı etkileşimini sürdürmeye ve artırmaya yardımcı olacaktır.

3. Sonuç:

Yani, Arama Sorguları Anomali Algılama, performans metriklerine göre aykırı olan sorguları belirlemek anlamına gelir. İşletmeler için beklenmedik şekilde yüksek veya düşük Tıklama Oranları, gibi potansiyel sorunları veya fırsatları tespit etmek değerlidir.